<a href="https://colab.research.google.com/github/pninav/StyleGan_FewShot/blob/main/StyleGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting to google-drive.

In [ ]:
from google.colab import drive
import os
drive.mount('/content/Pnina/')
os.chdir('/content/Pnina/MyDrive/')

Delete Project folder for fresh start.

In [ ]:
import shutil
shutil.rmtree('/content/Pnina/MyDrive/StyleGan_FewShot',ignore_errors=True)

Cloning StyleGan project (creates a local directory in google drive, initializes a. git directory inside it), and installing requirements. 

In [ ]:

!git clone https://github.com/pninav/StyleGan_FewShot.git
os.chdir('/content/Pnina/MyDrive/StyleGan_FewShot/')
!pip install -r requirements.txt



Download Kaggle dataset using Kaggle API. 

In [ ]:
import os
os.chdir('/content/Pnina/MyDrive/StyleGan_FewShot/')
os.environ['KAGGLE_CONFIG_DIR'] = '/content/Pnina/MyDrive/StyleGan_FewShot/kaggle/'
!mkdir -p kaggle_dataset
os.chdir('/content/Pnina/MyDrive/StyleGan_FewShot/kaggle_dataset')
!kaggle competitions download -c gan-getting-started 

Create StyleGan_Fewshot folder (the GitHub files will be transferred there).

In [ ]:
os.chdir('/content/Pnina/MyDrive/StyleGan_FewShot/')

Save the thirty images of Monet.

In [ ]:
!python save_images_monet.py

View selected Monet images.

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
from glob import glob
for file in glob('/content/Pnina/MyDrive/StyleGan_FewShot/kaggle_dataset/monet_reduced/*.*'):
  img = cv2.imread(file, cv2.IMREAD_UNCHANGED)
  cv2_imshow(img)


Download pretrained StyleGan.

In [ ]:
os.makedirs('./checkpoint',exist_ok=True)
os.chdir('/content/Pnina/MyDrive/StyleGan_FewShot/checkpoint')


Download the training weights.

In [ ]:
!gdown --id 1QlXFPIOFzsJyjZ1AtfpnVhqW4Z0r8GLZ
os.chdir('/content/Pnina/MyDrive/StyleGan_FewShot/')


Downloading...
From: https://drive.google.com/uc?id=1QlXFPIOFzsJyjZ1AtfpnVhqW4Z0r8GLZ
To: /content/Pnina/MyDrive/StyleGan_FewShot/checkpoint/stylegan-256px-new.model
725MB [00:12, 59.4MB/s]


Data preparation - produces one file of all the images.

In [ ]:
os.makedirs('./dataset',exist_ok=True)
!python prepare_data.py --out monet --dataset /content/Pnina/MyDrive/StyleGan_FewShot/kaggle_dataset/monet_reduced/


Pre-computing the MiFID score on the real photos (Monet images).  

In [ ]:
!python precompute_acts.py --dataset monet


Model training

After each Epoch, performs the calculations, including in MiFID that were previously calculated on the real images, to improve model performance.

In [ ]:
!python finetune.py --name monet --mixing --loss r1 --sched --dataset monet --freeze_D --feature_loc 3
